In [1]:
import bagpy
from bagpy import bagreader
import numpy as np
#%matplotlib # for interactive plots
from matplotlib import pyplot as plt

In [2]:
# This program computes landing metrics to see how accurate the landing was,
#   and how well the landing pad stayed in the FOV
# Record a rosbag during a flight with 
#   rosbag record /re_tag_detections /ground_truth/state

# For FOV constraint metric:
# /re_tag_detections (geometry_msgs/PoseWithCovarianceStamped) contains messages:
#   header
#   pose

In [ ]:
# Camera Info

# Simulation
fx = 317.0682951064176 # vert focal length in pixels
fy = 317.0682951064176 # horiz focal length in pixels
cx = 320.5 # x principal point in pixels
cy = 240.5 # y principal point in pixels

# Hardware
# fx =  # vert focal length in pixels
# fy =  # horiz focal length in pixels
# cx =  # x principal point in pixels
# cy =  # y principal point in pixels

In [6]:
# Load rosbag

#path = '/home/sequoyah/ROS/rosbags/'
path = '/home/sequoyah/Documents/Research/ARC_lab/VIO/rosbags/'
#path = '/filespace/w/walters/ROS/VIO_quad/rosbags/'

#bag_file = 'sim_thrust_map.bag'
#bag_file = '2023-07-21-03-51-16.bag'
bag_file_fov = 'FOV_2023-08-09-23-09-38.bag'
bag_file_no_fov = 'noFOV_2023-08-09-23-06-30.bag'

bag_fov = bagreader(path + bag_file_fov)
bag_no_fov = bagreader(path + bag_file_no_fov)
print("\nMessage types: ", bag_fov.message_types)
print("Message frequencies: ", bag_fov.frequency)

[INFO]  Successfully created the data folder /home/sequoyah/Documents/Research/ARC_lab/VIO/rosbags/FOV_2023-08-09-23-09-38.
[INFO]  Successfully created the data folder /home/sequoyah/Documents/Research/ARC_lab/VIO/rosbags/noFOV_2023-08-09-23-06-30.

Message types:  ['nav_msgs/Odometry', 'geometry_msgs/PoseStamped']
Message frequencies:  [62.50000000058975, 14.705882352944716]


In [9]:
# Convert to csv files

tag_detect_fov_csv = bag_fov.message_by_topic('/re_tag_detections')
tag_detect_no_fov_csv = bag_no_fov.message_by_topic('/re_tag_detections')
print(tag_detect_fov_csv,'\n', tag_detect_no_fov_csv)

/home/sequoyah/Documents/Research/ARC_lab/VIO/rosbags/FOV_2023-08-09-23-09-38/re_tag_detections.csv 
 /home/sequoyah/Documents/Research/ARC_lab/VIO/rosbags/noFOV_2023-08-09-23-06-30/re_tag_detections.csv


In [10]:
# Convert to numpy arrays

tag_fov_data = np.genfromtxt(tag_detect_fov_csv, delimiter=',', dtype=float, names=True)
tag_no_fov_data = np.genfromtxt(tag_detect_no_fov_csv, delimiter=',', dtype=float, names=True)

# Show field names
print(tag_fov_data.dtype.names)

('Time', 'headerseq', 'headerstampsecs', 'headerstampnsecs', 'headerframe_id', 'posepositionx', 'posepositiony', 'posepositionz', 'poseorientationx', 'poseorientationy', 'poseorientationz', 'poseorientationw')


In [11]:
# Extract data

# Times
times_fov = tag_fov_data['Time']
times_no_fov = tag_no_fov_data['Time']

# Position of landing pad wrt camera
x_fov = tag_fov_data['posepositionx']
y_fov = tag_fov_data['posepositiony']
z_fov = tag_fov_data['posepositionz']
x_no_fov = tag_no_fov_data['posepositionx']
y_no_fov = tag_no_fov_data['posepositiony']
z_no_fov = tag_no_fov_data['posepositionz']

In [ ]:
# Set plot params
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams['font.size'] = 15
#print(plt.rcParams.keys)

# show smoothing effect
plt.figure(0)
plt.plot(imu_times, imu_data['linear_accelerationx'], alpha=0.3)
plt.plot(imu_times, acc_x_smooth)
plt.xlabel('Time (sec)')
plt.ylabel('m/s^2')
plt.title('x acceleration')
plt.figure(1)
plt.plot(imu_times, imu_data['linear_accelerationy'], alpha=0.3)
plt.plot(imu_times, acc_y_smooth)
plt.xlabel('Time (sec)')
plt.ylabel('m/s^2')
plt.title('y acceleration')
plt.figure(2)
plt.plot(imu_times, imu_data['linear_accelerationz'], alpha=0.3)
plt.plot(imu_times, acc_z_smooth)
plt.xlabel('Time (sec)')
plt.ylabel('m/s^2')
plt.title('z acceleration')
plt.show()

In [ ]:
# Functions for rotating a 3-vector by a quaternion

def q_mult(q1, q0):
    # quaternion product q1xq0
    w0, x0, y0, z0 = q0
    w1, x1, y1, z1 = q1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)
def q_inv(q):
    # quaternion inverse
    return np.array([-q[0], q[1], q[2], q[3]])
def pnt2q(pnt):
    # represent a 3-vector as a quaternion
    return np.array([0, pnt[0], pnt[1], pnt[2]])
def q2pnt(q):
    # remove extra zero from rotated pnt
    assert abs(q[0]) < 1e-14, f"q[0] too big, got: {q[0]}" # make sure the removed value is about zero
    return np.array([q[1], q[2], q[3]])
def q_rotate(pnt, q):
    # rotate pnt by q
    result_q = q_mult(q_inv(q), q_mult( pnt2q(pnt),q))
    return q2pnt(result_q)

In [ ]:
# Estimate mass-normalized thrust

acc_w = np.empty(3)
g_vect = np.array([0,0,9.81])
mass_norm_thrust = np.empty(thrust_times.size)
mass_norm_thrust[:] = np.nan

for i in range(thrust_times.size):
    acc_w = q_rotate(acc[i,:], att[i,:]) # Express imu acceleration in world frame (values include gravity)
    mass_norm_thrust[i] = np.linalg.norm(acc_w) # Estimate mass-normalized thrust

In [ ]:
# Create plots

start_sec = thrust_times[0] + 15  # add 1 to start 1 second later
end_sec = thrust_times[-1] - 38   # subtract 2 to end 2 secs earlier
# select indices to satisfy desired time region, and desired altitutde
plt_idx = (thrust_times > start_sec)* \
          (thrust_times < end_sec)
#* \
#          (alt > min_alt)

plt.figure(3)
plt.plot(thrust_times[plt_idx], PX4thrust[plt_idx])
plt.xlabel('Time (sec)')
plt.ylabel('Thrust (PX4 normalized)')
plt.title('PX4 Normalized Thrust')

plt.figure(4)
plt.plot(thrust_times[plt_idx], mass_norm_thrust[plt_idx])
plt.xlabel('Time (sec)')
plt.ylabel('Thrust (mass normalized)')
plt.title('Mass Normalized Thrust')

# Display the plots
plt.show()

In [ ]:
# Fit polynomial to data

mnt_fit = mass_norm_thrust[plt_idx]
px4t_fit = PX4thrust[plt_idx]

# Append zeros
mnt_fit = np.append(mnt_fit, np.zeros(30))
px4t_fit = np.append(px4t_fit, np.zeros(30))

d, c, b, a = polyfit(mnt_fit,  px4t_fit, 3)
print([a,b,c,d])
#a,b,c,d = [0.00013850414341400538, -0.005408755617324549, 0.11336157680888627, -0.0022807568577082674]
x = np.linspace(0,20,100)

plt.figure(5)
plt.plot(mnt_fit, px4t_fit, '.')
plt.xlabel('Mass normalized thrust ($m/s^2$)')
plt.ylabel('PX4 normalized thrust')
plt.title('Thrust mapping')
plt.xlim(-0.35)
plt.ylim(-0.05,1.05)

# plot ploynomial
plt.plot(x, a*x*x*x + b*x*x + c*x + d)

# Display the plots
plt.show() # comment this out if using interactive plots